In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install openpyxl
!pip install gdown
!pip install farasapy

In [ ]:
!gdown 1h620Wmx1yvkTKibH6N2wCNddOakBTUzg -O data.xlsx
!git clone https://github.com/aub-mind/arabert

In [ ]:
from transformers import TFBertModel, BertTokenizer, BertConfig
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from arabert.preprocess import ArabertPreprocessor

arabert_prep = ArabertPreprocessor(model_name='aubmindlab/bert-base-arabertv02-twitter',keep_emojis=False)
MAX_LENGHT = 128
config = BertConfig.from_pretrained( 'aubmindlab/bert-base-arabertv02-twitter', output_hidden_states=False)    
pre = BertTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")
bert = TFBertModel.from_pretrained("aubmindlab/bert-base-arabertv02-twitter", config=config, from_pt=True, trainable=True)

In [ ]:
mf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/MSA-Train.csv')

mf.rename(columns = {'Query_MSA' : 'Query'}, inplace = True)
mf.rename(columns = {'Query_ID_MSA' : 'Query_ID'}, inplace = True)

mf.head()

In [ ]:
pf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/PAL-Train.csv')

pf.rename(columns = {'Query_PAL' : 'Query'}, inplace = True)
pf.rename(columns = {'Query_ID_PAL' : 'Query_ID'}, inplace = True)

pf.head()

In [ ]:
mf['dialect'] = 'MSA'

mf.head()

In [ ]:
pf['dialect'] = 'PAL'

pf.head()

In [ ]:
df = pd.concat([mf, pf], ignore_index = True)

df.head()

In [ ]:
df.shape

In [ ]:
classes = df['Intent_en'].unique().tolist()
print(classes)
len(classes)

In [ ]:
dialects = df['dialect'].unique().tolist()
print(dialects)
len(dialects)

In [ ]:
df1 = df

df1.head()

In [ ]:
mf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/MSA-Dev.csv')

mf.rename(columns = {'Query_MSA' : 'Query'}, inplace = True)
mf.rename(columns = {'Query_ID_MSA' : 'Query_ID'}, inplace = True)

mf.head()

In [ ]:
mf['dialect'] = 'MSA'

mf.head()

In [ ]:
pf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/PAL-Dev.csv')

pf.rename(columns = {'Query_PAL' : 'Query'}, inplace = True)
pf.rename(columns = {'Query_ID_PAL' : 'Query_ID'}, inplace = True)

pf.head()

In [ ]:
pf['dialect'] = 'PAL'

pf.head()

In [ ]:
df1 = pd.concat([df1, mf], ignore_index = True)

df1 = pd.concat([df1, pf], ignore_index = True)

In [ ]:
df1.shape

In [ ]:
df1['dialect'].unique().tolist()

In [ ]:
mf = pd.read_csv('/kaggle/input/arabic-nlp/data/Banking77_Arabized_MSA_test_sample.csv')

mf.rename(columns = {'text' : 'Query'}, inplace = True)

mf = mf.drop(['label'], axis = 1)

mf['dialect'] = 'MSA'

mf.head()

In [ ]:
pf = pd.read_csv('/kaggle/input/arabic-nlp/data/Banking77_Arabized_PAL_test_sample.csv')

pf.rename(columns = {'text' : 'Query'}, inplace = True)

pf = pf.drop(['label'], axis = 1)

pf['dialect'] = 'PAL'

pf.head()

In [ ]:
df1 = pd.concat([df1, mf], ignore_index = True)

df1 = pd.concat([df1, pf], ignore_index = True)

In [ ]:
df1.shape

In [ ]:
df1['Query'] = df1['Query'].apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df1["Query"], df1["dialect"],
                                                    stratify=df1["dialect"], 
                                                    test_size=0.2)

In [ ]:
#X = X.apply(lambda x: pre(x, return_tensors="tf"))
y_train = y_train.apply(lambda x: dialects.index(x))
y_valid = y_valid.apply(lambda x: dialects.index(x))

X_train = pre(X_train.tolist(), return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)
X_valid = pre(X_valid.tolist(), return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)
#print(pre_data)

In [ ]:

input_text = tf.keras.Input(shape=(MAX_LENGHT,), dtype=tf.int32, name="input")
#preprocessing = keras.layers.Lambda(lambda_layer, name="lambda_layer")(text.tolist())
bert_output = bert(input_text)

net = tf.keras.layers.Dropout(0.5, name='DropOut1')(bert_output['pooler_output'])
net = tf.keras.layers.Dense(units=768, activation='tanh', name='classifier')(net)
net = tf.keras.layers.Dropout(0.5, name='DropOut2')(net)
net = tf.keras.layers.Dense(units=len(dialects), activation='softmax', name='output')(net)

model = tf.keras.Model(input_text, net)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.keras.metrics.SparseCategoricalAccuracy(name="acc")

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              loss=loss,
              metrics=metrics
            )

In [ ]:
history = model.fit(
                    x = X_train['input_ids'],
                    y = y_train,
                    validation_data = (X_valid['input_ids'], y_valid),
                    epochs=20,
                   shuffle=True
                   )

In [ ]:
from matplotlib import pyplot as plt
print(history.history.keys())
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(len(loss_train))

plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('fig-loss.png')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
print(history.history.keys())
acc_train = history.history['acc']
acc_val = history.history['val_acc']
epochs = range(len(acc_train))

plt.plot(epochs, acc_train, 'g', label='Training Accuracy')
plt.plot(epochs, acc_val, 'b', label='validation Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('fig-Accuracy.png')
plt.show()

In [ ]:
pf.head()

In [ ]:
#print(classes)
example = "عندكم معلومات عن بطاقة الدفع عند الاستلام؟"
ids = pre(example, return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)['input_ids']
pred = model.predict(ids)
index = tf.math.argmax(pred[0])
print(dialects[index] + " with percentage: " + str(pred[0][index] * 100) + "\n")

scores = {}
for i in range(len(dialects)):
    scores[dialects[i]] = pred[0][i] * 100

scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
for k,v in scores.items():
    print(k + " with percentage: " + "{:.2f}".format(v))

In [ ]:
print(dialects[index])

In [ ]:
mf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/MSA-Train.csv')

mf.rename(columns = {'Query_MSA' : 'Query'}, inplace = True)
mf.rename(columns = {'Query_ID_MSA' : 'Query_ID'}, inplace = True)

mf.head()

In [ ]:
pf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/PAL-Train.csv')

pf.rename(columns = {'Query_PAL' : 'Query'}, inplace = True)
pf.rename(columns = {'Query_ID_PAL' : 'Query_ID'}, inplace = True)

pf.head()

In [ ]:
mf['Query'] = mf['Query'].apply(lambda x: arabert_prep.preprocess(x))
pf['Query'] = pf['Query'].apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_valid1, y_train1, y_valid1 = train_test_split(mf["Query"], mf["Intent_en"],
                                                    stratify=mf["Intent_en"], 
                                                    test_size=0.1)


X_train2, X_valid2, y_train2, y_valid2 = train_test_split(pf["Query"], pf["Intent_en"],
                                                    stratify=pf["Intent_en"], 
                                                    test_size=0.1)

In [ ]:
mf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/MSA-Dev.csv')

mf.rename(columns = {'Query_MSA' : 'Query'}, inplace = True)
mf.rename(columns = {'Query_ID_MSA' : 'Query_ID'}, inplace = True)

mf.head()

In [ ]:
pf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/PAL-Dev.csv')

pf.rename(columns = {'Query_PAL' : 'Query'}, inplace = True)
pf.rename(columns = {'Query_ID_PAL' : 'Query_ID'}, inplace = True)

pf.head()

In [ ]:
mf['Query'] = mf['Query'].apply(lambda x: arabert_prep.preprocess(x))
pf['Query'] = pf['Query'].apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
X_valid1 = pd.concat([X_valid1, mf['Query']], ignore_index=True)
print(X_valid1.shape)

X_valid2 = pd.concat([X_valid2, pf['Query']], ignore_index=True)
print(X_valid2.shape)

In [ ]:
y_valid1 = pd.concat([y_valid1, mf['Intent_en']], ignore_index=True)
print(y_valid1.shape)

y_valid2 = pd.concat([y_valid2, pf['Intent_en']], ignore_index=True)
print(y_valid2.shape)

In [ ]:
#X = X.apply(lambda x: pre(x, return_tensors="tf"))
y_train1 = y_train1.apply(lambda x: classes.index(x))
y_valid1 = y_valid1.apply(lambda x: classes.index(x))

X_train1 = pre(X_train1.tolist(), return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)
X_valid1 = pre(X_valid1.tolist(), return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)
#print(pre_data)

#X = X.apply(lambda x: pre(x, return_tensors="tf"))
y_train2 = y_train2.apply(lambda x: classes.index(x))
y_valid2 = y_valid2.apply(lambda x: classes.index(x))

X_train2 = pre(X_train2.tolist(), return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)
X_valid2 = pre(X_valid2.tolist(), return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)
#print(pre_data)

In [ ]:

input_text = tf.keras.Input(shape=(MAX_LENGHT,), dtype=tf.int32, name="input")
#preprocessing = keras.layers.Lambda(lambda_layer, name="lambda_layer")(text.tolist())
bert_output = bert(input_text)

net = tf.keras.layers.Dropout(0.5, name='DropOut1')(bert_output['pooler_output'])
net = tf.keras.layers.Dense(units=768, activation='tanh', name='classifier')(net)
net = tf.keras.layers.Dropout(0.5, name='DropOut2')(net)
net = tf.keras.layers.Dense(units=len(classes), activation='softmax', name='output')(net)

model1 = tf.keras.Model(input_text, net)
model2 = tf.keras.Model(input_text, net)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.keras.metrics.SparseCategoricalAccuracy(name="acc")

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              loss=loss,
              metrics=metrics
            )

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              loss=loss,
              metrics=metrics
            )

In [ ]:
history1 = model1.fit(
                    x = X_train1['input_ids'],
                    y = y_train1,
                    validation_data = (X_valid1['input_ids'], y_valid1),
                    epochs=30,
                   shuffle=True
                   )

In [ ]:
from matplotlib import pyplot as plt
print(history1.history.keys())
loss_train = history1.history['loss']
loss_val = history1.history['val_loss']
epochs = range(len(loss_train))

plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('fig-loss.png')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
print(history1.history.keys())
acc_train = history1.history['acc']
acc_val = history1.history['val_acc']
epochs = range(len(acc_train))

plt.plot(epochs, acc_train, 'g', label='Training Accuracy')
plt.plot(epochs, acc_val, 'b', label='validation Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('fig-Accuracy.png')
plt.show()

In [ ]:
history2 = model2.fit(
                    x = X_train2['input_ids'],
                    y = y_train2,
                    validation_data = (X_valid2['input_ids'], y_valid2),
                    epochs=30,
                   shuffle=True
                   )

In [ ]:
from matplotlib import pyplot as plt
print(history2.history.keys())
loss_train = history2.history['loss']
loss_val = history2.history['val_loss']
epochs = range(len(loss_train))

plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('fig-loss.png')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
print(history2.history.keys())
acc_train = history2.history['acc']
acc_val = history2.history['val_acc']
epochs = range(len(acc_train))

plt.plot(epochs, acc_train, 'g', label='Training Accuracy')
plt.plot(epochs, acc_val, 'b', label='validation Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('fig-Accuracy.png')
plt.show()

In [ ]:
intent = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/Intents-Dictionary.csv')

intent.head()

In [ ]:
mp = {}

for i in range(intent.shape[0]):
    mp[intent['Intent_en'][i]] = intent['Intent_ID'][i]

In [ ]:
kf = pd.read_csv('/kaggle/input/arabic-nlp/ArabicNLP/test/subtask1-test-set.csv')

kf.head()

In [159]:
preds = []

for i in range(0, 3000):
    example = kf['Query'][i]
    ids = pre(example, return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)['input_ids']
    pred = model.predict(ids)
    index = tf.math.argmax(pred[0])
    if(dialects[index] == 'MSA'):
        pred = model1.predict(ids)
        index = tf.math.argmax(pred[0])
    else:
        pred = model2.predict(ids)
        index = tf.math.argmax(pred[0])
#     print(classes[index] + " with percentage: " + str(pred[0][index] * 100) + "\n")
    preds.append(classes[index])
    print(i)    

1/1 [==============================] - 0s 40ms/step
0
1/1 [==============================] - 0s 41ms/step
1
1/1 [==============================] - 0s 40ms/step
2
1/1 [==============================] - 0s 53ms/step
3
1/1 [==============================] - 0s 38ms/step
4
1/1 [==============================] - 0s 41ms/step
5
1/1 [==============================] - 0s 55ms/step
6
1/1 [==============================] - 0s 66ms/step
7
1/1 [==============================] - 0s 42ms/step
8
1/1 [==============================] - 0s 43ms/step
9
1/1 [==============================] - 0s 45ms/step
10
1/1 [==============================] - 0s 42ms/step
11
1/1 [==============================] - 0s 45ms/step
12
1/1 [==============================] - 0s 40ms/step
13
1/1 [==============================] - 0s 75ms/step
14
1/1 [==============================] - 0s 41ms/step
15
1/1 [==============================] - 0s 40ms/step
16
1/1 [==============================] - 0s 42ms/step
17
1/1 [===============

KeyboardInterrupt: 

In [ ]:
for i in range(3000, 6000):
    example = kf['Query'][i]
    ids = pre(example, return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)['input_ids']
    pred = model.predict(ids)
    index = tf.math.argmax(pred[0])
    if(dialects[index] == 'MSA'):
        pred = model1.predict(ids)
        index = tf.math.argmax(pred[0])
    else:
        pred = model2.predict(ids)
        index = tf.math.argmax(pred[0])
#     print(classes[index] + " with percentage: " + str(pred[0][index] * 100) + "\n")
    preds.append(classes[index])
    print(i)

In [ ]:
for i in range(6000, 9000):
    example = kf['Query'][i]
    ids = pre(example, return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)['input_ids']
    pred = model.predict(ids)
    index = tf.math.argmax(pred[0])
    if(dialects[index] == 'MSA'):
        pred = model1.predict(ids)
        index = tf.math.argmax(pred[0])
    else:
        pred = model2.predict(ids)
        index = tf.math.argmax(pred[0])
#     print(classes[index] + " with percentage: " + str(pred[0][index] * 100) + "\n")
    preds.append(classes[index])
    print(i)


In [ ]:
for i in range(9000, 11721):
    example = kf['Query'][i]
    ids = pre(example, return_tensors="tf", padding='max_length', max_length=MAX_LENGHT, truncation=True)['input_ids']
    pred = model.predict(ids)
    index = tf.math.argmax(pred[0])
    if(dialects[index] == 'MSA'):
        pred = model1.predict(ids)
        index = tf.math.argmax(pred[0])
    else:
        pred = model2.predict(ids)
        index = tf.math.argmax(pred[0])
#     print(classes[index] + " with percentage: " + str(pred[0][index] * 100) + "\n")
    preds.append(classes[index])
    print(i)


In [ ]:
len(preds)

In [ ]:
iids = []
for i in range(len(preds)):
    iids.append(mp[preds[i]])

In [ ]:
kf = kf.drop(['Query'], axis = 1)

kf['IntentID'] = iids

kf.head()

In [ ]:
kf.to_csv('/kaggle/working/Fired_from_NLP_subtask1_test_pred_3.csv', index=False)